In [1]:
import pm4py
import os

In [2]:
log = pm4py.read_xes(os.path.join("data", "BPI_Challenge_2019-3-w-after.xes.gz"))
#log = pm4py.read_xes(os.path.join("data", "endoint-variation-new.xes.gz"))

parsing log, completed traces ::   0%|          | 0/15182 [00:00<?, ?it/s]

In [4]:
bpmn_graph = pm4py.read_bpmn("./bpmn/BPMN_Celonis_as-is_final.bpmn"
                             )

net, im, fm = pm4py.convert_to_petri_net(bpmn_graph)

# Token-based replay

In [ ]:
replayed_traces = pm4py.conformance_diagnostics_token_based_replay(log, net, im, fm)

replaying log with TBR, completed variants ::   0%|          | 0/4228 [00:00<?, ?it/s]

In [ ]:
missing_tokens = 0
consumed_tokens = 0
remaining_tokens = 0
produced_tokens = 0

for trace in replayed_traces:
    missing_tokens += trace["missing_tokens"]
    consumed_tokens += trace["consumed_tokens"]
    remaining_tokens += trace["remaining_tokens"]
    produced_tokens += trace["produced_tokens"]

first_eq = 0.5 * (1 - float(missing_tokens)/float(consumed_tokens))
second_eq = 0.5 * (1 - float(remaining_tokens)/float(produced_tokens))

fitness = first_eq + second_eq

str((fitness, missing_tokens, consumed_tokens, remaining_tokens, produced_tokens))

# Alignments

In [10]:
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments

aligned_traces = alignments.apply_log(log, net, im, fm)

from pm4py.algo.evaluation.replay_fitness import algorithm as replay_fitness
log_fitness = replay_fitness.evaluate(aligned_traces, variant=replay_fitness.Variants.ALIGNMENT_BASED)

print(log_fitness)

aligning log, completed variants ::   0%|          | 0/9 [00:00<?, ?it/s]

{'percFitTraces': 100.0, 'averageFitness': 1.0, 'percentage_of_fitting_traces': 100.0, 'average_trace_fitness': 1.0, 'log_fitness': 0.9998695850925249}


# Footprints

In [12]:
from pm4py.algo.discovery.footprints import algorithm as footprints_discovery
fp_log = footprints_discovery.apply(log, variant=footprints_discovery.Variants.ENTIRE_EVENT_LOG)
fp_trace_by_trace = footprints_discovery.apply(log, variant=footprints_discovery.Variants.TRACE_BY_TRACE)
fp_net = footprints_discovery.apply(net, im, fm)

KeyboardInterrupt: 

In [ ]:
from pm4py.visualization.footprints import visualizer as fp_visualizer

gviz = fp_visualizer.apply(fp_net, parameters={fp_visualizer.Variants.SINGLE.value.Parameters.FORMAT: "svg"})
fp_visualizer.view(gviz)

In [ ]:
conf_fp = pm4py.conformance_diagnostics_footprints(fp_trace_by_trace, fp_net)

In [ ]:
from pm4py.algo.conformance.footprints import algorithm as fp_conformance
from pm4py.algo.conformance.footprints.util import evaluation

conf_result = fp_conformance.apply(fp_log, fp_net, variant=fp_conformance.Variants.LOG_EXTENSIVE)

fitness = evaluation.fp_fitness(fp_log, fp_net, conf_result)
precision = evaluation.fp_precision(fp_log, fp_net)